In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
data_release='2021-04-27'

df = pd.read_csv(join("..", "data", "1.1.resurgence", "case rate and severity risk", "case_numbers.csv"))

df = df.rename(columns={
    "n.all": "n_all", 
    "n.severe": "n_severe"
})

df = df.drop(columns=['Unnamed: 0', 'n_severe'])

COUNTRY = ['ALL', 'FRANCE', 'GERMANY', 'ITALY', 'SPAIN', 'USA']
COUNTRY_COLOR = ['white', '#0072B2', '#E79F00', '#029F73', '#57B4E9', '#D45E00']

# df.country = df.country.apply(lambda x: x.capitalize())

df

In [ ]:
d = df.copy()

plot = alt.Chart(
    d
).mark_bar(
    size=35,
#     stroke='black',
#     strokeWidth=1
).encode(
    x=alt.X("wave:N", title=None, axis=alt.Axis(labels=False)),
    y=alt.Y("n_all:Q", title=None, scale=alt.Scale(domain=[0, 45000])),
    color=alt.Color("wave:N", scale=alt.Scale(range=['#D45E00', '#0072B2']), title='Wave'),
).properties(
    width=100,
    height=200
)

text = plot.mark_text(
    size=16, dx=0, dy=-5, color='white', baseline='bottom', fontWeight=500
).encode(
#     x=alt.X('month:N'),
#     y=alt.Y('value:Q', stack='zero'),
    x=alt.X("wave:N", title=None, axis=alt.Axis(labels=False)),
    y=alt.Y("n_all:Q", title=None),
#     detail='cat:N',
    text=alt.Text('n_all:Q'),#, format='.0%'),
#     order="order:O",
#     opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]))
)

plot = (plot + text).facet(
    column=alt.Column("country:N", header=alt.Header(title=None), sort=COUNTRY)
).resolve_scale(color='shared')

plot = plot.properties(
    title={
        "text": [
            f"Country-Level Cases By Wave"
        ],
        "dx": 45,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ],
        "subtitleColor": "gray",
    }
)


# plot = alt.vconcat(*(
#     plot_lab(df=df, lab=lab) for lab in unique_sites
# ), spacing=30)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom',
    legend_title_orient='left',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=100
)

plot

In [ ]:
df = pd.read_csv(join("..", "data", "1.1.resurgence", "case rate and severity risk", "percCountCountry_AndAll_Weeks.csv"), sep='\t')

df = df.rename(columns={
    "Country": "country", 
    "weeks": "week"
})

# df = df.drop(columns=['Unnamed: 0', 'n_severe'])

df.country = df.country.apply(lambda x: x.upper())

COUNTRY = ['ALL', 'FRANCE', 'GERMANY', 'ITALY', 'SPAIN', 'USA']
COUNTRY_COLOR = ['black', '#0072B2', '#E79F00', '#029F73', '#57B4E9', '#D45E00']

df

In [ ]:
d = df.copy()

plot = alt.Chart(
    d
).mark_line(
    size=2.5,
    point=alt.OverlayMarkDef(filled=True, strokeWidth=4, opacity=0.7),
    opacity=0.7
).encode(
    x=alt.X("week:O", title=None, axis=alt.Axis(labelAngle=0, tickCount=0)),
    y=alt.Y("percentage:Q", title=None, axis=alt.Axis(format=".0%")),
    color=alt.Color("country:N", scale=alt.Scale(domain=COUNTRY, range=COUNTRY_COLOR), title='Country'),
).properties(
    width=650,
    height=350
)

# text = plot.mark_text(
#     size=16, dx=0, dy=-5, color='white', baseline='bottom', fontWeight=500
# ).encode(
# #     x=alt.X('month:N'),
# #     y=alt.Y('value:Q', stack='zero'),
#     x=alt.X("wave:N", title=None, axis=alt.Axis(labels=False)),
#     y=alt.Y("percentage:Q", title=None, scale=alt.Scale(format=".1%")),
# #     detail='cat:N',
#     text=alt.Text('n_all:Q'),#, format='.0%'),
# #     order="order:O",
# #     opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]))
# )

# plot = (plot).facet(
#     row=alt.Row("country:N", header=alt.Header(title=None), sort=COUNTRY)
# ).resolve_scale(color='shared')

plot = plot.properties(
    title={
        "text": [
            f"Case Rate Over Time By Country"
        ],
        "dx": 35,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ],
        "subtitleColor": "gray",
    }
)


# plot = alt.vconcat(*(
#     plot_lab(df=df, lab=lab) for lab in unique_sites
# ), spacing=30)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='top-right',
    legend_title_orient='top',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=30
)

plot

In [ ]:
df = pd.read_csv(join("..", "data", "1.1.resurgence", "case rate and severity risk", "severity_shift_random_effects.csv"))

df = df.rename(columns={
    "ci.lwr": "ci_l", 
    "ci.upr": "ci_u",
    "n.all": "n",
    "weeks": "week"
})

df = df[df.effect_size == 'risk_ratio']

df['e_l'] = df.pp - df.se
df['e_u'] = df.pp + df.se

df['sig'] = df['sig'].apply(lambda x: 'p<0.05' if x == True else 'False')

COUNTRY = ['ALL', 'FRANCE', 'GERMANY', 'ITALY', 'SPAIN', 'USA']
COUNTRY_COLOR = ['black', '#0072B2', '#E79F00', '#029F73', '#57B4E9', '#D45E00']

df

In [ ]:
d = df.copy()

d = d[d.day == 0]

plot = alt.Chart(
    d
).mark_point(
    size=180,
    filled=True,
    shape='diamond',
#     point=alt.OverlayMarkDef(filled=True, strokeWidth=4, opacity=0.7),
    opacity=1
).encode(
    y=alt.Y("country:O", title=None, axis=alt.Axis(labelAngle=0, tickCount=0, labels=True)),
    x=alt.X("pp:Q", title=None, scale=alt.Scale(zero=False)),
    color=alt.Color("country:N", scale=alt.Scale(domain=COUNTRY, range=COUNTRY_COLOR), legend=None, title='Country'),
#     stroke=alt.Stroke("sig", scale=alt.Scale(domain=['p<0.05'], range=['black']), title='Significance')
).properties(
    width=300,
    height=300
)

guide = alt.Chart(
    pd.DataFrame({'baseline': [1, 1, 1, 1, 1, 1], 'country': COUNTRY})
).mark_rule(color='gray', strokeDash=[3,3], opacity=0.5).encode(
    x=alt.X('baseline:Q')
)
    
tick = plot.mark_errorbar(
    opacity=0.7, color='black'
).encode(
    y=alt.Y("country:O", title='Days Since Admission'),
    x=alt.X("ci_l:Q"),
    x2=alt.X2("ci_u:Q"),
    stroke=alt.value('black')
)

plot = (tick + plot + guide)
# .facet(
#     row=alt.Row("country:N", header=alt.Header(title=None, labelAngle=0, labelAnchor='start', labelPadding=30), sort=COUNTRY)
# ).resolve_scale(color='shared')

plot = plot.properties(
    title={
        "text": [
            f"Early To Late Severity Risk Ratios At Admission"
        ],
        "dx": 60,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ],
        "subtitleColor": "gray",
    }
)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='right',
    legend_title_orient='top',
    axis_label_font_size=14,
    header_label_font_size=16
)

plot